In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
# Load the dataset
# Replace 'dataset.csv' with the actual filename or path.
data = pd.read_csv('depression_dataset.csv', encoding='latin-1')

In [5]:
# Separate the text data and labels
X_text = data['text'].values
y_labels = data['label'].values

In [6]:
# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_labels)

In [7]:
# Convert the integer labels to one-hot encoded vectors
y_onehot = to_categorical(y_encoded)

In [8]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_text)
X_sequences = tokenizer.texts_to_sequences(X_text)

In [9]:
# Pad sequences to ensure they have the same length
max_sequence_length = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

In [10]:
# Define the bidirectional model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(64)))  # You can adjust the number of LSTM units
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Print the model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 310, 100)          7140300   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dense (Dense)               (None, 4)                 516       
                                                                 
Total params: 7,225,296
Trainable params: 7,225,296
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_onehot, test_size=0.2, random_state=42)

In [14]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
272/994 [=======>......................] - ETA: 13:08 - loss: 0.3919 - accuracy: 0.8611

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Generate accuracy and loss graphs
plt.figure(figsize=(12, 6))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()